In [1]:
import pandas as pd
import requests
import unicodedata
from bs4 import BeautifulSoup as soup

In [2]:
raw_path = '../raw'    

In [8]:
df = pd.read_csv(raw_path + '/' + 'ehrenamtskartei-gnd.csv', usecols=['nachname','vorname','umfang','geburt','tod','idns'] )
df

,nachname,vorname,umfang,geburt,tod,idns
0,Adrian,Werner,NaN,21.04.1915,18.08.2001,['133012964']
1,Anrich,Gerold,NaN,24.11.1942,04.10.2013,[]
2,Arndt,Martin,NaN,02.10.1917,02.03.2005,[]
3,Assmus,Ursula,NaN,05.06.1921,26.02.2017,['116371943']
4,Bader,Thomas,NaN,1942,25.03.2014,[]
...,...,...,...,...,...,...
499,Zahrnt,Heinz,NaN,31.05.1915,01.11.2003,['11863609X']
500,Zill,Wolfgang,NaN,10.12.1928,16.06.1986,[]
501,Zipp,Marianne,NaN,14.03.1920,01.10.1983,[]
502,Zirk,Heinz,NaN,21.01.1947,09.08.2012,[]


# GND-Suche

In [4]:

base_url = 'https://services.dnb.de/sru/authorities?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [18]:

def gnd_abfrage(row):
    params.update({'query': f'PER = "{row.nachname}, {row.vorname}" AND BBG=Tp*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content)    
    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None

            # exaktes datum datx finden
            try:
                datx = record.find('subfield', {'code': '4'}, string="datx").parent.find('subfield', {'code': 'a'}).string
            except:
                datx = ''
            
            # jahresdatum in 100 finden
            try:
                datl_100 = record.find('datafield', {'tag': '100'}).find('subfield', {'code': 'd'}).string
            except:
                datl_100 = ''
            
            # jahresdatum in 548 datl finden
            try:
                datl_548 = record.find('subfield', {'code': '4'}, string="datl").parent.find('subfield', {'code': 'a'}).string
            except:
                datl_548 = ''

            #setzen von datum_match, wenn geburtsjahre übereinstimmen

            if datx == row.lebensdaten:
                record_match['datum_match'] = True
            elif datl_100.split('-')[0] == row.geburt[-4] and datl_100.split('-')[1] == row.tod[-4]:
                record_match['datum_match'] = True
            elif datl_548.split('-')[0] == row.geburt[-4] and datl_548.split('-')[1] == row.tod[-4]:
                record_match['datum_match'] = True
            else:
                record_match['datum_match'] = False

            matches.append(record_match)
        
        return matches
    except Exception as e:
        return "fehler %r" % e

In [ ]:
def gnd_auswertung(matches):
    return [match['idn'] for match in matches if match['datum_match'] == True]

In [19]:
# idn abfrage starten und ergebnisse in neue spalte matches schreiben
df['matches'] = df.apply(gnd_abfrage, axis=1)

In [24]:
# auswertung starten und matchende idns in neue spalte idns schreiben
df['idns'] = df.matches.apply(gnd_auswertung)